## Calculate $\pi$ with IPython Parallel

A simple Monte Carlo simulation to approximate the value of  $\pi$  involves randomly selecting points in a unit square and determining how many of them land in $x^2+y^2=1$. 



Start the ipcluster in a terminal with 4 engines:
```
    $ source pythonhpc.sh 
    $ ipcluster start --n 4 &
```
Then import ipyparallel in your notebook, and initialize a Client instance:

In [1]:
import ipyparallel as ipp
client = ipp.Client()
client

Create a DirectView object for direct execution on the engines:

In [16]:
dview = client[:]
dview

<DirectView [0, 1, 2, 3,...]>

Parallelize the evaluation of $pi$ using a Monte Carlo method. First load modules, and export the random module to the engines: 

In [18]:
from random import random
from math import pi
dview['random'] = random

The function monte_carlo_pi is a Monte Carlo method to calculate $\pi$. We will time the execution of this function using %timeit -n 1 and a sample size of 10 million. This is running in serial.

In [4]:
def monte_carlo_pi(nsamples):
    s = 0
    for i in range(nsamples):
        x = random()
        y = random()
        if x*x + y*y <= 1:
            s+=1
    return 4.*s/nsamples

In [22]:
%%timeit -n 1 
pi_mc = monte_carlo_pi(10000000)
# should take a couple of seconds per timeit trial
print(pi_mc)

3.141332
3.1411688
3.1413156
3.1408932
3.1422228
3.1419028
3.141344
2.09 s ± 43.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<mark>Excercise</mark>: The incomplete function below should take a DirectView object and a number of samples, divide the number of samples between the engines, and call monte_carlo_pi() with a subset of the samples on each engine. 

Complete the parallel function (by replacing the "TO_DO" fields), call it with $10^7$ samples, time it and compare the performance with the serial version.

In [17]:
dview.targets

[0, 1, 2, 3, 4, 5, 6, 7]

In [19]:
def multi_monte_carlo_pi(view, nsamples):
    p = len(view.targets)
    if nsamples % p:
        # to ensure even divisibility
        nsamples += p - (nsamples%p)
    
    subsamples = nsamples//p
    
    ar = view.apply(monte_carlo_pi, subsamples)
    return sum(ar)/p

In [20]:
%%timeit -n 5
multi_monte_carlo_pi(dview, 10000000)

322 ms ± 5.39 ms per loop (mean ± std. dev. of 7 runs, 5 loops each)


<mark>Question</mark> Confirm that the results of the serial and parallel pi calculation are correct.

<mark>Question</mark> Did you see any speedup? 

Add another 2 engines by running 

```
$ ipengine &
```

twice in your terminal. Ensure that your view includes the additional engines. Run the parallel $\pi$ calculation again. Do you see any further speedup? 

<mark>Question</mark> Finally add another 2 engines and test again. Have you reached the limit of scalability on this toy problem?
